In [2]:
#!pip install segmentation_models
#%env SM_FRAMEWORK=tf.keras

In [1]:
import numpy as np
import pandas as pd
import os
from IPython.display import display
from glob import glob
import matplotlib.pyplot as plt
import math

import segmentation_models as sm

import tensorflow as tf
from sklearn.model_selection import train_test_split

Segmentation Models: using `tf.keras` framework.


In [4]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

ValueError: Please provide a TPU Name to connect to.

In [2]:
class CFG:
    debug=True

    dataset_dir="../input/ranzcr-clip-catheter-line-classification/" if debug \
        else "gs://kds-8e853601547c2d2e7a243638123e66bb32174c3a2b50da35aca5baed/"
    annotated_tfrec="../input/cvc_annotated_mask.tfrec" if debug else "gs://km_kaggle_catheter/cvc_annotated_mask.tfrec"
    tfrec_length=8853
    batch_size=2 if debug else tpu_strategy.num_replicas_in_sync*16 
    input_size=(576,576)
    epochs=1 if debug else 30

if not CFG.debug:
    from kaggle_datasets import KaggleDatasets

In [3]:
train_size=0.8
train_length=math.floor(CFG.tfrec_length*train_size)

In [17]:
feature_description={
    "StudyInstanceUID":tf.io.FixedLenFeature([],tf.string),
    "image":tf.io.FixedLenFeature([],tf.string)
}

def _parse_function(record):
    return tf.io.parse_example(record,feature_description)

AUTO=tf.data.experimental.AUTOTUNE

def decode_img(file_bytes):
    image=tf.io.decode_jpeg(file_bytes,channels=1)
    image=tf.image.resize(image,CFG.input_size)
    image/=255.0
    return image

def preprocess(record):
    annotated_img=decode_img(record["image"])
    uid=record["StudyInstanceUID"]
    original_path=f"{CFG.dataset_dir}train/"+uid+".jpg"
    original_img=decode_img(tf.io.read_file(original_path))
    return original_img,annotated_img

def build_dataset(train_length):
    dset=tf.data.TFRecordDataset([CFG.annotated_tfrec])
    return dset.take(train_length),dset.skip(train_length)

def dset_postprocess(dset,shuffle=256):
    dset=dset.map(_parse_function,num_parallel_calls=AUTO)
    dset=dset.map(preprocess,num_parallel_calls=AUTO)
    dset=dset.cache()
    dset=dset.shuffle(shuffle) if shuffle else dset
    dset=dset.batch(CFG.batch_size).prefetch(AUTO)
    return dset

train_dset,val_dset=build_dataset(train_length)
train_dset=dset_postprocess(train_dset)
val_dset=dset_postprocess(val_dset)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [18]:
def create_model():
    model=sm.PSPNet(classes=1,activation="sigmoid",encoder_weights=None,input_shape=CFG.input_size+(1,))
    model.compile(optimizer="adam",loss=sm.losses.bce_jaccard_loss,metrics=[sm.metrics.iou_score])
    return model

if not CFG.debug:
    with tpu_strategy.scope():
        model=create_model()
else:
    model=create_model()

In [19]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
ckpt=tf.keras.callbacks.ModelCheckpoint("unet_ckpt",save_best_only=True,monitor="val_iou_score",
                                        mode="max",options=save_locally)
model.fit(train_dset,epochs=CFG.epochs,validation_data=val_dset,callbacks=[ckpt])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

KeyboardInterrupt: 